In [7]:
import os
import logging

import pandas as pd
import numpy as np
from scipy import stats

In [5]:
# Seaborn and matplotlib visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')                    
sns.set_style({'axes.grid':False}) 

# Plotly visualization
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected = True)

In [11]:
logging.basicConfig(format="%(asctime)s [%(name)s] [%(levelname)s] %(message)s")
logger = logging.getLogger(name="JPX")
logger.setLevel(logging.INFO)

def info(msg):
    logger.info(msg)
    
def warn(msg):
    logger.warn(msg)
    
def error(msg):
    logger.error(msg)

## Load Data

In [12]:
DATA_BASE_DIR = "./data/"

TRAIN_DIRS = [
    os.path.join(DATA_BASE_DIR, "train_files")
]

info(f"train dirs: {TRAIN_DIRS}")

2022-04-16 00:24:49,289 [JPX] [INFO] train dirs: ['./data/train_files']
